<a href="https://colab.research.google.com/github/sanujsriv/text-data-preprocessing/blob/dummy/Preprocessing_OnlyDocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing Steps :- 
1. Convert to lowercase
2. Remove punctuations with empty space 
3. Remove digits
4. Apply lemmatization
5. Remove Stopwords
6. Remove words that do not have word embeddings
7. Remove words that have length < 3.
*** NOT APPLYING STEMMING

Generating Preprocessed Docs Steps:
1. Apply Preprocessing Steps
2. Remove Documents that have length < 3
3. Apply CountVectorizer Data transform { with min_df =3 i.e. words that appear in less than 3 documents are removed }
4. Remove documents that are empty after countvectorization
5. Remove words in documents that do not appear in the vocab generated by countvectorization
6. Generate embeddings using the vocab 


# Just Run these

In [ ]:
#@title function : load / save pickle_obj
import pickle
import csv
import pickle5
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def load_obj_pkl5(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle5.load(f)

In [ ]:
#@title Imports, wordnet,punkt
import nltk
import os
import glob
import pandas as pd
# nltk.download('stopwords')  
nltk.download('punkt')
nltk.download('wordnet')

import pandas as pd
import re
import string
from time import time
import numpy as np
import collections
from sklearn.feature_extraction.text import CountVectorizer
from numpy import random
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import os
import pandas
if 'grad16' in os.getcwd(): local=True
else: local = False

In [ ]:
#@title If local run this...
# if local == True:
#   colab_dir= 'C:\\Users\\sanuj\\Documents\\FoTo\\colab'
#   os.chdir(colab_dir)
#   directory= 'Preprocessing'
#   try : os.mkdir(directory)
#   except FileExistsError:pass
#   os.chdir(directory)

# Preprocessing [StopWords,Stemming,Punctuations] & Word2Vec 

In [ ]:
stem = PorterStemmer()
wnl = WordNetLemmatizer()
stopwords = ['la','wa','will','fa','ha','pa','co','v','said','cant','better','well','going','will','would','know','dont','get','like','think','im',"also","said","a", "able", "about", "above", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "after", "afterwards", "again", "against", "ah", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "auth", "available", "away", "awfully", "b", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "between", "beyond", "biol", "both", "brief", "briefly", "but", "by", "c", "ca", "came", "can", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "could", "couldnt", "d", "date", "did", "didn't", "different", "do", "does", "doesn't", "doing", "done", "don't", "down", "downwards", "due", "during", "e", "each", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "for", "former", "formerly", "forth", "found", "four", "from", "further", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "had", "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "hed", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hers", "herself", "hes", "hi", "hid", "him", "himself", "his", "hither", "home", "how", "howbeit", "however", "hundred", "i", "id", "ie", "if", "i'll", "im", "immediate", "immediately", "importance", "important", "in", "inc", "indeed", "index", "information", "instead", "into", "invention", "inward", "is", "isn't", "it", "itd", "it'll", "its", "itself", "i've", "j", "just", "k", "keep  keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "more", "moreover", "most", "mostly", "mr", "mrs", "much", "mug", "must", "my", "myself", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "now", "nowhere", "o", "obtain", "obtained", "obviously", "of", "off", "often", "oh", "ok", "okay", "old", "omitted", "on", "once", "one", "ones", "only", "onto", "or", "ord", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "re", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "s", "said", "same", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "she", "shed", "she'll", "shes", "should", "shouldn't", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure    t", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'll", "theyre", "they've", "think", "this", "those", "thou", "though", "thoughh", "thousand", "throug", "through", "throughout", "thru", "thus", "til", "tip", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "very", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "was", "wasnt", "way", "we", "wed", "welcome", "we'll", "went", "were", "werent", "we've", "what", "whatever", "what'll", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "whose", "why", "widely", "willing", "wish", "with", "within", "without", "wont", "words", "world", "would", "wouldnt", "www", "x", "y", "yes", "yet", "you", "youd", "you'll", "your", "youre", "yours", "yourself", "yourselves", "you've", "z", "zero"]
 
def preprocessing(doc,word2vec_model,my_punctuation,min_word_length=0):
    doc = doc.lower()
    doc = doc.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))  # string.punctuation[6] = ' won't --> won , don't --> don
    doc = word_tokenize(doc)    
    doc = filter(lambda x: x not in my_punctuation, doc)  
    doc = filter(lambda x:not x.isdigit(), doc)
    doc = [wnl.lemmatize(w.lower()) for w in doc]
    #doc = [stem.stem(w) for w in doc]
    doc = filter(lambda x:x not in stopwords, doc)
    doc = filter(lambda x: x in word2vec_model.vocab or x in ".",doc)
    doc = ' '.join(e for e in doc if len(e)>=min_word_length)
    # doc = ' '.join(e for e in doc)
    return doc

In [ ]:
from gensim import models
if local==True: 
  if not os.path.isfile("/home/grad16/sakumar/miniconda3/bin/GoogleNews-vectors-negative300.bin"):
    !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    !gunzip GoogleNews-vectors-negative300.bin.gz
  word2vec_model = models.KeyedVectors.load_word2vec_format('/home/grad16/sakumar/miniconda3/bin/GoogleNews-vectors-negative300.bin', binary=True)

if local==False:
  # https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O GoogleNews-vectors-negative300.bin.gz && rm -rf /tmp/cookies.txt
  !gunzip GoogleNews-vectors-negative300.bin.gz
  word2vec_model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Data --> Corpus & Labels

In [ ]:
# %cd /content
# !rm -r *

# get_Data Functions

## get_corpus_labels

In [ ]:
def get_yahoo_answers(dtype):
  gdrive_fileid = """0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU"""
  gdrive_filename = """yahoo_answers_csv.tar.gz"""

  if local:
    home_dir = '/home/grad16/sakumar/colab/preprocessing'
  if not local:
    home_dir = '/content'

  dir= home_dir+'/yahoo_answers_csv'
  if not os.path.exists(dir) and not os.path.isfile(gdrive_filename):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU" -O yahoo_answers_csv.tar.gz && rm -rf /tmp/cookies.txt
    os.system("tar -xvzf yahoo_answers_csv.tar.gz")
  # os.makedirs(home_dir+dir,exist_ok=True)
  os.chdir(dir)
  dict_labels = {'1':'Society & Culture', '2': 'Science & Mathematics', 
                '3': 'Health', '4': 'Education & Reference', '5' : 'Computers & Internet',
                '6': 'Sports', '7': 'Business & Finance', '8': 'Entertainment & Music',
                '9': 'Family & Relationships' , '10':'Politics & Government'}

  labels = []
  corpus = []
  files = ['train.csv','test.csv']
  # files = ['train.csv']
  count = 0
  for f in files:
    with open(f, newline='') as csvfile:
        reader = csv.reader(csvfile)
  #       # next(reader)
        if dtype == 'short':
          for row in reader:
            count = count+1
            labels.append(dict_labels[row[0]])
            # corpus.append(' '.join(t for t in row[1:]))
            corpus.append(' '.join(t for t in row[1:3])) # SHORT
        elif dtype == "full": 
          for row in reader:
            count = count+1
            labels.append(dict_labels[row[0]])
            corpus.append(' '.join(t for t in row[1:]))
      
  os.chdir(home_dir)
  return corpus,labels


In [ ]:
# def sog_news()

In [ ]:
# def get_dbpedia()

In [ ]:
def get_twentynews():
  from sklearn.datasets import fetch_20newsgroups
  data_20news=fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))
  targets = data_20news.target
  target_labels = data_20news.target_names 
  labels = [target_labels[t] for t in targets]
  return data_20news.data , labels

In [ ]:
def get_NewsCategory():
  os.system("wget -N https://www.dropbox.com/s/irvy3tdd305nhlr/news_category.zip")
  os.system("unzip News_Category.zip")
  df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
  labels = df['category'].values
  corpus = df['headline'].values+" . "+df['short_description'].values
  return corpus,labels

In [ ]:
def get_AGNews_data_120k():
  # try : os.mkdir(data_to_get)
  # except FileExistsError:pass
  # os.chdir(data_to_get)
  if not os.path.isfile('ag_news_csv.tar.gz'):
    os.system('wget -N -c https://www.dropbox.com/s/tyzue51quuo5y79/ag_news_csv.tar.gz')
    os.system('tar -xvzf ag_news_csv.tar.gz')
  if local:
    home_dir = '/home/grad16/sakumar/colab/preprocessing'
    dir= home_dir+'/ag_news_csv'
    os.chdir(dir)
    # os.chdir(colab_dir+'\\'+directory+'\\'+'AGNews')
  else:
    os.chdir('/content/ag_news_csv')
  dict_labels = {'1':'World', '2': 'Sports', '3': 'Business', '4': 'Sci/Tech'}
  labels = []
  corpus = []
  # files = ['train.csv','test.csv']
  files = ['train.csv']
  count = 0
  for f in files:
    with open(f, newline='') as csvfile:
        reader = csv.reader(csvfile)
        # next(reader)
        for row in reader:
          count = count+1
          labels.append(row[0])
          corpus.append(row[-1])
        # corpus = corpus[1:]
        # labels = labels[1:]
  labels = [dict_labels[l] for l in labels]
  os.chdir('..')
  return corpus,labels

In [ ]:
def get_AGNews_data():
  # try : os.mkdir(data_to_get)
  # except FileExistsError:pass
  # os.chdir(data_to_get)
  # # os.system('wget -N -c https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/b0f38b56-b494-4074-9dfd-d80825cac772  -O WebOfScience.zip')
  # os.system('unzip WebOfScience.zip')
  if local:
    os.chdir(colab_dir+'\\'+directory+'\\'+'AGNews')
  dict_labels = {'1':'World', '2': 'Sports', '3': 'Business', '4': 'Sci/Tech'}
  labels = []
  corpus = []
  files = ['train.csv','test.csv']
  count = 0
  for f in files:
    with open(f, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:
          count = count+1
          labels.append(row[0])
          corpus.append(row[-1])
        # corpus = corpus[1:]
        # labels = labels[1:]
  labels = [dict_labels[l] for l in labels]
  os.chdir('..')
  return corpus,labels

In [ ]:
def get_WOS46985_data(dtype):
    # try : os.mkdir(data_to_get)
    # except FileExistsError:pass
    # os.chdir(data_to_get)
  os.system('wget -N -c https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/b0f38b56-b494-4074-9dfd-d80825cac772  -O WebOfScience.zip')
  os.system('unzip WebOfScience.zip')
  
  if local: os.chdir(colab_dir+'\\'+directory+'\\'+'WebOfScience\\WOS46985')
  if local ==False: os.chdir('/content/WOS46985')
  
  dict_labels = {'0':'Computer  Science','1':'Electrical  Engineering', '2': 'Psychology', '3': 'Mechanical  Engineering',
  '4': 'Civil  Engineering' , '5': 'Medical  Science', '6': 'biochemistry'}
  if dtype=='full':  
    with open('X.txt','r') as f:
      content = f.readlines()
      content = [x.strip().lower() for x in content]
      corpus = content 
  elif dtype=='short':
    os.chdir('/content/Meta-data')
    WOS_keywords = np.array(pd.read_excel('Data.xlsx')['keywords'])
    WOS_keywords = [w.strip() for w in WOS_keywords]
    WOS_keywords = [w.split(';') for w in WOS_keywords]
    WOS_keywords = [''.join(e) for e in WOS_keywords]
    WOS_keywords = [w.strip() for w in WOS_keywords]
    corpus = WOS_keywords
  if local ==False: os.chdir('/content/WOS46985')
  with open('YL1.txt','r') as f:
      content = f.readlines()
      content = [x.strip().lower() for x in content]
      content = [dict_labels[x] for x in content]
      labels = content
  os.chdir('..')
  return corpus,labels

In [ ]:
def get_5ktweets_data():
  os.system('wget -N https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv')
  labels = []
  corpus = []
  with open('full-corpus.csv', newline='') as csvfile:
      reader = csv.reader(csvfile)
      for row in reader:
        labels.append(row[0])
        corpus.append(row[-1])
      corpus = corpus[1:]
      labels = labels[1:]
      return corpus,labels

In [ ]:
def get_bbc_data(dtype='full'):
  os.system('wget -N https://www.dropbox.com/s/vunli21d312x55g/bbc.zip')
  os.system('wget -N https://www.dropbox.com/s/h6y9zfdb76gl4uz/bbc-fulltext.zip')
  os.system('unzip bbc.zip')
  os.system('unzip bbc-fulltext.zip')

  # BBC Docs -
  corpus = []
  subfolders = [f.path for f in os.scandir(os.getcwd()+'/bbc') if f.is_dir()]
  subfolders = sorted(subfolders)
  for s in subfolders:
    files_list = sorted(glob.glob(s+"/*.txt")) 
    for file in files_list:
      with open(file, "rb") as f:
        content = f.readlines()
        if dtype == 'short':
          content = [content[0],content[2]] # headlines and abstracts 
        content = [x.strip().lower().decode('ISO-8859-1') for x in content] 
        corpus.append(' '.join(content).strip())

  # BBC_labels -
  with open("bbc.classes", "r") as f:
    content = f.readlines()
    content = [x.strip()[::-1][0] for x in content] 
    labels = content[4:]
    label_dict = {'0':'business','1':'entertainment','2':'politics','3':'sport','4':'tech'}
  for l in range(len(labels)):
    labels[l] = label_dict[labels[l]]
  return corpus,labels

In [ ]:
def get_searchsnippet_data():
  os.system('wget http://jwebpro.sourceforge.net/data-web-snippets.tar.gz')
  os.system('tar -xvzf data-web-snippets.tar.gz')
  corpus = []
  labels=[]
  with open("/content/data-web-snippets/train.txt", "r") as f:
    content = f.readlines()
    content = [x.split('\n')[0] for x in content]
    labels_train = [x.split(' ')[-1] for x in content]
    content = [' '.join(x.split(' ')[:-1]) for x in content]
    corpus.extend(content)
    labels.extend(labels_train)

  with open("/content/data-web-snippets/test.txt", "r") as f:
    content = f.readlines()
    content = [x.split('\n')[0] for x in content]
    labels_test = [x.split(' ')[-1] for x in content]
    content = [' '.join(x.split(' ')[:-1]) for x in content]
    corpus.extend(content)
    labels.extend(labels_test)
  return corpus,labels

In [ ]:
def get_stackoverflow_data():  
  data_labels_dict = {1: 'wordpress', 2: 'oracle', 3: 'svn',4: 'apache', 5: 'excel',6: 'matlab',7: 'visual-studio',8: 'cocoa',9: 'osx',10: 'bash',11: 'spring',12: 'hibernate',13: 'scala',14: 'sharepoint',15: 'ajax',16: 'qt',17: 'drupal',18: 'linq',19: 'haskell',20: 'magento'}
  os.system('wget -N https://raw.githubusercontent.com/jacoxu/StackOverflow/master/rawText/label_StackOverflow.txt')
  os.system('wget -N https://raw.githubusercontent.com/jacoxu/StackOverflow/master/rawText/title_StackOverflow.txt')
  corpus = []
  labels=[]
  with open("/content/title_StackOverflow.txt", "r") as f:
      content = f.readlines()
      content = [x.split('\n')[0] for x in content]
      corpus.extend(content)
  with open("/content/label_StackOverflow.txt", "r") as f:
      content = f.readlines()
      content = [x.split('\n')[0] for x in content]
      # content = [data_labels_dict[int(x)] for x in content]
      labels.extend(content)
  return corpus,labels

## get_Data_Driver

In [ ]:
def get_data(data_to_get,dtype):
  data_to_get = data_to_get.lower()
  if data_to_get == 'bbc':
    corpus,labels = get_bbc_data(dtype)
  elif data_to_get == 'searchsnippet':
    corpus,labels = get_searchsnippet_data()
  elif data_to_get == 'stackoverflow':
    corpus,labels = get_stackoverflow_data()
  elif data_to_get == '5ktweets':
    corpus,labels = get_5ktweets_data()
  elif data_to_get == 'wos':
    corpus,labels = get_WOS46985_data(dtype)
  elif data_to_get == 'agnews':
    corpus,labels = get_AGNews_data()
  elif data_to_get == 'newscategory':
    corpus,labels = get_NewsCategory()
  elif data_to_get == 'agnews120k':
    corpus,labels = get_AGNews_data_120k()  
  elif data_to_get == 'twentynews':
    corpus,labels = get_twentynews()  
  elif data_to_get == 'yahooanswers':
    corpus,labels = get_yahoo_answers(dtype)
  return corpus,labels

## doc label preprocessing | random sampling

In [ ]:
# def lookUpWord(vec,dtm,word):
#     i = vec.get_feature_names().index(word)
#     return dtm[:,i].nonzero()[0]

# from sklearn.feature_extraction.text import CountVectorizer
# import numpy as np

# corpus = [
# 'This is the first document.',
# 'This is the second second document.',
# 'And third one.',
# 'Is this the first document?'
# ]

# X = CountVectorizer(max_features=2)
# Y = X.fit_transform(corpus)
# # lookUpWord(X,Y,'first')

In [ ]:
def vocab_filtered_data(doc,vocab):
  doc = word_tokenize(doc)
  doc = filter(lambda x: x in vocab, doc) 
  doc = ' '.join(e for e in doc)
  return doc

def docs_labels_preprocessing(docs,labels,word2vec_model,min_doc_len,min_word_length,max_features):
  data_preprocessed = []
  data_preprocessed_labels = []
  embeddings = {}

  for i in range(len(docs)):
    doc = preprocessing(docs[i],word2vec_model,string.punctuation,min_word_length)
    data_preprocessed.append(doc)
    data_preprocessed_labels.append(labels[i])
 
  vectorizer = CountVectorizer(max_features=max_features,dtype=np.float32)
  train_vec = vectorizer.fit_transform(data_preprocessed).toarray()
  vocab = vectorizer.vocabulary_

  nonzeros_indexes = np.where(train_vec.any(1))[0]
  data_preprocessed = [data_preprocessed[i] for i in nonzeros_indexes]
  data_preprocessed_labels = [data_preprocessed_labels[i] for i in nonzeros_indexes]

  for i in range(len(data_preprocessed)):
    data_preprocessed[i] = vocab_filtered_data(data_preprocessed[i],vocab) 

  data_preprocessed_f = [data_preprocessed[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]
  data_preprocessed_labels_f = [data_preprocessed_labels[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]
  
  vectorizer_f = CountVectorizer(dtype=np.float32)
  vectorizer_f.fit_transform(data_preprocessed_f)
  vocab_f = vectorizer.vocabulary_

  for f in vocab:
    embeddings[f] = word2vec_model[f]
  return data_preprocessed_f,data_preprocessed_labels_f,embeddings,vocab_f

In [ ]:
def docs_random_sample(data_preprocessed,data_preprocessed_labels,sample_size):
  ## BBC - 250 random docs sampling (50 from each label)
  
  df = pd.DataFrame(data_preprocessed)
  df['classes'] = data_preprocessed_labels
  df = df.groupby('classes').apply(lambda x: x.sample(sample_size))
  index = list(df.index)
  ret_index  = np.asarray([i[1] for i in index])

  data_prepocessed_sampled= df.values[:,0]
  data_prepocessed_labels_sampled = df.values[:,1]
  # index = np.random.choice(df.values[:,0].shape[0], sample_size*len(set(data_preprocessed_labels)), replace=False)  
  # data_prepocessed_sampled = data_prepocessed_sampled[index]
  # data_prepocessed_labels_sampled = data_prepocessed_labels_sampled[index]
  return data_prepocessed_sampled,data_prepocessed_labels_sampled,ret_index

# **DATA GENERATION**

In [ ]:
# %cd /content/
# !rm -r *

In [ ]:
## Driver - Data Generation
# all_data_to_get = ['bbc','searchsnippet','wos'] # bbc,searchsnippet,stackoverflow,5ktweets,wos(Web Of Science)
                                                  # agnews,agnews120k,twentynews,yahooanswers
all_data_to_get = ['yahooanswers']
dtypes = ['short']

In [ ]:
#@title Short Data gen
def short_data_gen(data_preprocessed,data_preprocessed_labels,embeddings,f):
  os.system('mkdir -p /content/data_'+data_to_get+"/short")
  os.chdir('/content/data_'+data_to_get+"/short")

  data_preprocessed_short = []
  data_preprocessed_labels_short = []

  for doc,label in zip(data_preprocessed,data_preprocessed_labels):
    if len(doc.split(' ')) <= short_len_doc:
      data_preprocessed_short.append(doc)
      data_preprocessed_labels_short.append(label)

  vectorizer = CountVectorizer(dtype=np.float32)
  train_vec = vectorizer.fit_transform(data_preprocessed_short).toarray()
  vocab_short = vectorizer.vocabulary_

  f.write('\n\nlen of - \n  data_preprocessed_short: '+str(len(data_preprocessed_short))+'\n  data_preprocessed_labels_short: '+str(len(data_preprocessed_labels_short))+'\n  vocab_short:  '+str(len(vocab_short))+'\n')
  len_docs_short = [len(d.split(" ")) for d in data_preprocessed_short]
  f.write('\n min,mean,max short docs len: '+str(np.min(len_docs_short))+', '+str(np.mean(len_docs_short).round(2))+', '+str(np.max(len_docs_short))+'\n\n')
  f.write('**'*50)
  f.write("\n")

  save_obj(data_preprocessed_short,'data_preprocessed'+'_'+data_to_get+'_short')
  save_obj(data_preprocessed_labels_short,'data_preprocessed_labels'+'_'+data_to_get+'_short')
  save_obj(vocab_short,'vocab'+'_'+data_to_get+'_short') 
  save_obj(embeddings,'embeddings'+'_'+data_to_get+'_short')  

  os.chdir('/content/data_'+data_to_get+"/"+dtype)
  return data_preprocessed_short,data_preprocessed_labels_short,vocab_short,f

In [ ]:
## Settings :- 
min_doc_len = 3
min_word_length = 3
short_len_doc = 21

if local:
  home_dir = '/home/grad16/sakumar/colab/preprocessing'
elif not local:
  home_dir = '/'
# min_df=3
# if data_to_get == 'bbc': max_features = 2000
# elif data_to_get == 'searchsnippet': max_features = 3000
# elif data_to_get == 'wos': max_features = 4000

for data_to_get in all_data_to_get:

  if data_to_get == 'bbc': max_features = 2000
  elif data_to_get == 'searchsnippet': max_features = 3000
  elif data_to_get == 'wos': max_features = 4000
  elif data_to_get == 'newscategory': max_features = 4000
  elif data_to_get == 'agnews120k': max_features = 8000
  elif data_to_get == 'yahooanswers': max_features = 8000
  else: max_features = 4000
  
  for dtype in dtypes:
    if data_to_get=='searchsnippet' and dtype=='full':
      continue

    # os.chdir("/content")
    docs,labels = get_data(data_to_get,dtype)
    os.makedirs(home_dir+'/content/data_'+data_to_get+"/"+dtype,exist_ok=True)
    data_to_get

    with open(data_to_get+'_'+dtype+".txt", "a") as f:
      f.write(data_to_get+" - "+dtype)
      f.write("\n\n")
      f.write('**'*50)
      f.write("\n\n")
      f.write(str(len(docs))+', '+str(len(labels))+'\n')
      f.write('(labels,count): '+str(list(zip(*np.unique(labels, return_counts=True)))))
      data_preprocessed,data_preprocessed_labels,embeddings,vocab = docs_labels_preprocessing(docs,labels,word2vec_model,min_doc_len,min_word_length,max_features)
      f.write('\n\nlen of - \n  data_preprocessed: '+str(len(data_preprocessed))+'\n  data_preprocessed_labels: '+str(len(data_preprocessed_labels))+'\n  vocab:  '+str(len(vocab))+'\n  embeddings : '+str(len(embeddings))+'\n')
      len_docs = [len(d.split(" ")) for d in data_preprocessed]
      f.write('\n min,mean,max docs len: '+str(np.min(len_docs))+', '+str(np.mean(len_docs).round(2))+', '+str(np.max(len_docs))+'\n\n')
      
      save_obj(data_preprocessed,'data_preprocessed'+'_'+data_to_get+'_'+dtype)
      save_obj(data_preprocessed_labels,'data_preprocessed_labels'+'_'+data_to_get+'_'+dtype)   
      save_obj(vocab,'vocab'+'_'+data_to_get+'_'+dtype)
      save_obj(embeddings,'embeddings'+'_'+data_to_get+'_'+dtype)

      f.write('**'*50)

      ### short ####

      # data_preprocessed_short,data_preprocessed_labels_short,vocab_short,f = short_data_gen(data_preprocessed,data_preprocessed_labels,embeddings,f)

      ######

      # sample_size =20 # 20,50
      # name = 'small'
      # f.write(data_to_get+'_'+dtype+'_'+name+'\n\n\n\n:')
      # for i in range(1):
      #   id = str(i+1)
      #   d_data= data_to_get
      #   f.write('\n\nid : '+id+'\n\n')

      #   ## Small
      #   data_prepocessed_sampled,data_prepocessed_labels_sampled,index = docs_random_sample(data_preprocessed,data_preprocessed_labels,sample_size)
      #   f.write("len of data_preprocesssed_sampled: "+str(len(data_prepocessed_sampled)))
      #   f.write('\n(labels,count): '+str(list(zip(*np.unique(data_prepocessed_labels_sampled, return_counts=True)))))
      #   len_docs = [len(d.split(" ")) for d in data_prepocessed_sampled]
      #   vectorizer = CountVectorizer(dtype=np.float32)
      #   train_vec_sampled = vectorizer.fit_transform(data_prepocessed_sampled).toarray()
      #   vocab_sampled = vectorizer.vocabulary_
      #   f.write('\n min,mean,max docs len: '+str(np.min(len_docs))+', '+str(np.mean(len_docs).round(2))+', '+str(np.max(len_docs)))
      #   f.write('\n sampled_data_vocab_size: '+str(len(vocab_sampled)))

      #   save_obj(data_prepocessed_sampled,id+'_docs_sampled_'+name+'_'+str(sample_size)+d_data+dtype)
      #   save_obj(data_prepocessed_labels_sampled,id+'_labels_sampled_'+name+'_'+str(sample_size)+d_data+dtype)
      #   save_obj(index,id+'_index_'+name+'_'+str(sample_size)+d_data+dtype)
      # os.chdir(home_dir)

KeyboardInterrupt: ignored

In [ ]:
# os.chdir("/content")
# from google.colab import files
os.chdir(home_dir)
data_to_get = 'yahooanswers'
zipped_pickle_filename = data_to_get
if local==True:
  os.system('zip -r '+zipped_pickle_filename+'_'+str(max_features)+'_.zip '+'./content/data_'+data_to_get+"/"+dtype)
elif local==False:
  os.system('zip -r '+zipped_pickle_filename+'_'+str(max_features)+'_.zip '+'/content/data_'+data_to_get+"/"+dtype)

# files.download(zipped_pickle_filename+'_'+str(max_features)+'_.zip')

# os.system('zip -r '+zipped_pickle_filename+'_'+str(max_features)+'_'+str(sample_size)+'.zip /content/data_'+data_to_get)
# files.download(zipped_pickle_filename+'_'+str(max_features)+'_'+str(sample_size)+'.zip')

# os.system("rm -r *")

# Sampling of data

In [ ]:
%cd /content
!rm -r *

/content


In [ ]:
!unzip yahooanswers_8000_.zip

Archive:  yahooanswers_8000_.zip
   creating: content/data_yahooanswers/short/
  inflating: content/data_yahooanswers/short/yahooanswers_short.txt  
   creating: content/data_yahooanswers/short/.ipynb_checkpoints/
  inflating: content/data_yahooanswers/short/data_preprocessed_labels_yahooanswers_short.pkl  
  inflating: content/data_yahooanswers/short/data_preprocessed_yahooanswers_short.pkl  
  inflating: content/data_yahooanswers/short/embeddings_yahooanswers_short.pkl  
  inflating: content/data_yahooanswers/short/vocab_yahooanswers_short.pkl  


In [ ]:
# !pip install pickle5
# import pickle5

In [ ]:
# !wget https://www.dropbox.com/s/77a814l596kdz8b/yahooanswers_4000_short_complete.zip
# !unzip yahooanswers_4000_short_complete.zip
os.chdir(home_dir+'/content/data_'+data_to_get+"/"+dtype)

In [ ]:
pwd

'/home/grad16/sakumar/colab/preprocessing/content/data_yahooanswers/short'

In [ ]:
data_preprocessed = load_obj_pkl5('data_preprocessed_yahooanswers_short')
data_preprocessed_labels = load_obj_pkl5('data_preprocessed_labels_yahooanswers_short')

In [ ]:
pwd

'/home/grad16/sakumar/colab/preprocessing/content/data_yahooanswers/short'

In [ ]:
home_dir = '/home/grad16/sakumar/colab/preprocessing'
os.chdir(home_dir)

In [ ]:
# %cd /content
# !rm -r *

sample_size = 5000 # 20,50
# home_dir =''
all_data_to_get = ['yahooanswers']
dtypes = ['short']

data_to_get = all_data_to_get[0]
dtype = dtypes[0]

os.chdir(home_dir)
os.makedirs(home_dir+'/content/data_'+data_to_get+"/"+dtype+'/sample',exist_ok=True)
os.chdir(home_dir+'/content/data_'+data_to_get+"/"+dtype+'/sample')

with open(data_to_get+'_'+dtype+".txt", "a") as f:
  f.write(data_to_get+'_'+dtype+'_\n\n\n\n:')
  for i in range(1):
    id = str(i+1)
    d_data= data_to_get
    ## Small
    data_prepocessed_sampled,data_prepocessed_labels_sampled,index = docs_random_sample(data_preprocessed,data_preprocessed_labels,sample_size)
    f.write("len of data_preprocesssed_sampled: "+str(len(data_prepocessed_sampled)))
    f.write('\n(labels,count): '+str(list(zip(*np.unique(data_prepocessed_labels_sampled, return_counts=True)))))
    len_docs = [len(d.split(" ")) for d in data_prepocessed_sampled]
    vectorizer = CountVectorizer(dtype=np.float32)
    train_vec_sampled = vectorizer.fit_transform(data_prepocessed_sampled).toarray()
    vocab_sampled = vectorizer.vocabulary_
    embeddings_sampled ={}
    for v in vocab_sampled:
      embeddings_sampled[v] = word2vec_model[v]

    f.write('\n min,mean,max docs len: '+str(np.min(len_docs))+', '+str(np.mean(len_docs).round(2))+', '+str(np.max(len_docs)))
    f.write('\n sampled_data_vocab_size: '+str(len(vocab_sampled)))

    save_obj(data_prepocessed_sampled,'data_preprocessed'+'_'+data_to_get+'_'+dtype+'_'+str(sample_size))
    save_obj(data_prepocessed_labels_sampled,'data_preprocessed_labels'+'_'+data_to_get+'_'+dtype+'_'+str(sample_size))
    save_obj(vocab_sampled,'vocab'+'_'+data_to_get+'_'+dtype+'_'+str(sample_size))
    save_obj(embeddings_sampled,'embeddings'+'_'+data_to_get+'_'+dtype+'_'+str(sample_size))
    save_obj(index,'index_'+data_to_get+"_"+dtype+'_'+str(sample_size))
  
  # save_obj(data_prepocessed_sampled,id+'_docs_sampled_'+name+'_'+str(sample_size)+d_data+dtype)
  # save_obj(data_prepocessed_labels_sampled,id+'_labels_sampled_'+name+'_'+str(sample_size)+d_data+dtype)
  # save_obj(index,id+'_index_'+name+'_'+str(sample_size)+d_data+dtype)

# os.chdir('/content')
os.chdir(home_dir)

In [ ]:
data_prepocessed_sampled

array(['doe rogers school district school today',
       'renter utility doe generally utility included rent',
       'sex man grew breast', ...,
       'consider best player time broken surface lot work answer player win big match player multiple time man grass man clay man hard court double dream team match win marble woman',
       'join wwe doe sounded good write sounded good fight punk jeff hardy',
       'winner age religion'], dtype=object)

In [ ]:
np.asarray(data_preprocessed)[index]

array(['doe rogers school district school today',
       'renter utility doe generally utility included rent',
       'sex man grew breast', ...,
       'consider best player time broken surface lot work answer player win big match player multiple time man grass man clay man hard court double dream team match win marble woman',
       'join wwe doe sounded good write sounded good fight punk jeff hardy',
       'winner age religion'], dtype='<U2128')

In [ ]:
from google.colab import files
zipped_pickle_filename = data_to_get+'_sampled_'+str(sample_size)
if local: 
  os.system('zip -r '+zipped_pickle_filename+'.zip ./content/data_'+data_to_get+"/"+dtype+'/sample') 
if not local:
  os.system('zip -r '+zipped_pickle_filename+'_sampled_'+str(sample_size)+'.zip '+home_dir+'/content/data_'+data_to_get+"/"+dtype)
files.download(zipped_pickle_filename+'_'+str(sample_size)+'_sampled.zip')